<a href="https://colab.research.google.com/github/rohith033/NeuralArt/blob/main/styletransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(input image ,output image(first it would be noise) ,style image ) -> loss function =  loss(input,output) + loss(out,style)

optimize the output image so grads repured only for output image 

network arch 
---
vgg 19 without dense layer <br>
we woulde be taking intermediate outputs specifiaclly layers before pooling 
loss will be sum of output of all these layers 








the intermediate outputs will be the layer right after maxpolling they are listed below

now run it through the model and grad the weigths

preprocessing making shapes of image to 150 150 30

make class for model

In [67]:
import torch.nn as nn
from torchvision.models import vgg19
from PIL import Image
from torchvision import transforms
import torch
import torchvision

In [68]:
class v19(nn.Module):
  def __init__(self):
    super(v19,self).__init__()
    self.model=vgg19(pretrained=True).features
    self.outputs = ['0', '5', '10', '19', '28'] 
  def forward(self,x):
    output = []
    for itr , layer in self.model._modules.items():
      # print(itr)
      x  = layer(x)
      if itr in self.outputs:
        output.append(x)
    return output

In [69]:
preprocess =  transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor()
])
def load(path):
  image = Image.open(path)
  image = preprocess(image).unsqueeze(0)
  return image


In [70]:
art = load('./art.png')
input = load('./input.png')
styled = input.clone().requires_grad_(True)
v19=v19().eval()

In [71]:
art_weigth = v19(art)
input_weigth = v19(input)


In [72]:
print(len(art_weigth))

5


In [73]:
optimizer = torch.optim.Adam([styled], lr=0.001)

In [74]:
alpha = 1
beta = 0.1
itr = 300


In [75]:
for i in range(0,itr):
  styled_weigth = v19(styled)
  input_loss = 0
  art_loss = 0
  for l1,l2,l3 in zip(input_weigth,art_weigth,styled_weigth):    
    input_loss = torch.mean((l1-l3)**2)/(150*150*3)
    _, c, h, w = l2.size()
    l2 = l2.view(c, h * w)
    l3 = l3.view(c, h * w)
    art_loss = torch.mean((torch.mm(l2,l2.t())-torch.mm(l3,l3.t()))**2)/(150*150*3)
  loss = art_loss * beta + input_loss * alpha
  optimizer.zero_grad()
  loss.backward(retain_graph=True)
  optimizer.step()
  if(i%30==0):
      img = styled.clone().squeeze()
      torchvision.utils.save_image(img, 'output-{}.png'.format(i+1))
